In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 29.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import jiwer

#script

In [4]:
from sklearn.metrics import mean_absolute_error
import numpy as np

In [5]:
gt = pd.read_csv('/content/drive/MyDrive/хаки/ORG/RF/open_part/GT_train.csv',sep=';')
submission = pd.read_csv('/content/drive/MyDrive/хаки/ORG/RF/open_part/GT_train.csv',sep=';')

In [6]:
def calc_sub_metric(submission, gt):

  if len(submission['Транскрибированный текст'])!=len(gt['Транскрибированный текст']):
    raise Exception("Необходимо, чтобы submission.csv содержал все встречи из тестового датасета")

  if len(set(submission['Наименование аудиозаписи'].unique()) & set(gt['Наименование аудиозаписи'].unique()))!=len(gt):
    raise Exception('Столбец id в submission должен иметь те же значения, что и столбец id в grounded true')

  try:
      df = gt.merge(submission, how = "left", on = 'Наименование аудиозаписи', suffixes=('_true','_pred'))
  except Exception:
      assert False, "Ошибка при сравнении grounded_true и submission"


  df['WER'] = [1-jiwer.wer(x[0],x[1]) for x in zip(df['Транскрибированный текст_true'], df['Транскрибированный текст_pred'])]

  return df

In [7]:
df = calc_sub_metric(submission, gt)

In [8]:
df

,Наименование аудиозаписи,Транскрибированный текст_true,Число спикеров_true,Транскрибированный текст_pred,Число спикеров_pred,WER
0,Встреча 2,Апелляционный суд посредством видеоконференцсв...,2,Апелляционный суд посредством видеоконференцсв...,2,1.0
1,Встреча 4,Продолжаем проработку новых национальных проек...,1,Продолжаем проработку новых национальных проек...,1,1.0
2,Встреча 5,Столь представительный состав нашего сегодняшн...,2,Столь представительный состав нашего сегодняшн...,2,1.0
3,Встреча 8,"Михаил Владимирович, Вы знаете, что в Государс...",2,"Михаил Владимирович, Вы знаете, что в Государс...",2,1.0
4,Встреча 9,"Да, здравствуйте, я, Аня и Егор. Сегодня мы на...",3,"Да, здравствуйте, я, Аня и Егор. Сегодня мы на...",3,1.0
5,Встреча 10,"Доброе утро, уважаемые коллеги, как отметил пр...",2,"Доброе утро, уважаемые коллеги, как отметил пр...",2,1.0


In [9]:
def custom_metric(df, gt):

  mean_WER = df['WER'].sum()/len(gt)

  custom_MAE = 1/(mean_absolute_error(df['Число спикеров_true'].values, df['Число спикеров_pred'].values)+1)

  print(f'mean_WER : {mean_WER}')
  print(f'custom_MAE : {custom_MAE}')

  return mean_WER * custom_MAE


In [10]:
custom_metric(df, gt)

mean_WER : 1.0
custom_MAE : 1.0


1.0

#Симуляция ответов и тесты с метрикой

In [11]:
gt = pd.read_csv('/content/drive/MyDrive/хаки/ORG/RF/open_part/GT_train.csv',sep=';')
submission = pd.read_csv('/content/drive/MyDrive/хаки/ORG/RF/open_part/GT_train.csv',sep=';')

In [12]:
import random

def remove_random_words(text, percentage=0.5):
    words = text.split()
    num_words_to_remove = int(len(words) * percentage)
    indices_to_remove = random.sample(range(len(words)), num_words_to_remove)
    words = [word for i, word in enumerate(words) if i not in indices_to_remove]
    return ' '.join(words)

In [13]:
#Отсекаем часть от датасета для удобства перепроверки
mini_submission = submission.iloc[:2]
mini_gt = gt.iloc[:2]

#Искажаем правильные ответы
mini_submission['Транскрибированный текст'] = mini_submission['Транскрибированный текст'].apply( lambda x : remove_random_words(x, percentage=0.1)) # 10% слов удаляются
mini_submission['Число спикеров'] = mini_submission['Число спикеров'].apply( lambda x : max(1,x + random.randint(-1,1))) # отклонение от истинного значения на 1

<ipython-input-13-4476775b96b1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mini_submission['Транскрибированный текст'] = mini_submission['Транскрибированный текст'].apply( lambda x : remove_random_words(x, percentage=0.1)) # 10% слов удаляются
<ipython-input-13-4476775b96b1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mini_submission['Число спикеров'] = mini_submission['Число спикеров'].apply( lambda x : max(1,x + random.randint(-1,1))) # отклонение от истинного значения на 1


In [14]:
mini_submission

,Наименование аудиозаписи,Транскрибированный текст,Число спикеров
0,Встреча 2,Апелляционный суд видеоконференцсвязи рассматр...,3
1,Встреча 4,Продолжаем проработку новых национальных проек...,1


In [15]:
df = calc_sub_metric(mini_submission, mini_gt)

In [16]:
df

,Наименование аудиозаписи,Транскрибированный текст_true,Число спикеров_true,Транскрибированный текст_pred,Число спикеров_pred,WER
0,Встреча 2,Апелляционный суд посредством видеоконференцсв...,2,Апелляционный суд видеоконференцсвязи рассматр...,3,0.900114
1,Встреча 4,Продолжаем проработку новых национальных проек...,1,Продолжаем проработку новых национальных проек...,1,0.901004


In [17]:
final_metric = custom_metric(df, mini_gt)
final_metric

mean_WER : 0.9005589057693342
custom_MAE : 0.6666666666666666


0.6003726038462227